# Calculating the GBT time required to detect a galaxy cluster with MUSTANG-2

## Authors: Charles Romero and Emily Moravec

Last updated: 2024-12-30.

## Usage
You can use this notebook as a template to calculate the required GBT time to detect a cluster of a given mass and at a given redshift with MUSTANG-2. You will need to install M2_ProposalTools (see https://m2-tj.readthedocs.io/en/latest/index.html for details). This notebook is based on the "Example case of estimating observing time for assumed [A10](https://ui.adsabs.harvard.edu/abs/2010A%26A...517A..92A/abstract) clusters" example on M2_ProposalTools (https://m2-tj.readthedocs.io/en/latest/SinglePointing_M2.html which can be downloaded at https://github.com/CharlesERomero/M2_TJ/blob/master/docs/source/SinglePointing_M2.ipynb). 

## About
Hello potential MUSTANG-2 observer that wants to know how much telescope time is required to detect a given a cluster of a specific mass at a particular redshift at a particular signal-to-noise ratio. This notebook walks you through creating an expected compton-y signal map for your given parameters ($z$ and $M_{500}$), running your map through the MUSTANG-2 transfer functions to take into account the filtering of MUSTANG-2, and convolving this model map with the MUSTANG-2 beam. It then calculates the required observing time for your cluster based on your SNR requirement and the peak of the simulated SZ signal from the physical parameters you give it ($z$ and $M_{500}$).

## Note
The time estimates calculated in this notebook are made assuming using the MIDAS pipeline for data processing.

## Imports and constants

In [1]:
import M2_ProposalTools.WorkHorse as WH
import numpy as np
import astropy.units as u
import M2_ProposalTools.FilterImages as FI
import scipy

In [2]:
# Constants needed for calculations - it is recommended that you keep these default parameters.
h70            = 1                          # Normalization of Hubble parameter
fwhm           = 10.0                       # Roughly the resolution of MUSTANG-2
pixsize        = 2.0                        # Map pixel size (arcseconds) - making a map - package does beam convolution and filtering
s2f            = np.sqrt(8.0*np.log(2.0))   # Conversion between FWHM and sigma
pix_sigma      = fwhm/(pixsize*s2f)         # Gaussian sigma, in pixel size
y2k            = -3.3                       # Approximate conversion of compton-y to Kelvin. -3.3 is conservative whereas -3.4 is perhaps more accurate.

## MUSTANG-2 Parameters
Change these parameters based on your science. Consult https://gbtdocs.readthedocs.io/en/latest/references/receivers/mustang2/mustang2_mapping.html for the corresponding row.

In [3]:
scansize  = 3.5 # Scan size: 3' or 3.5' is recommended 
mapping_speed_uJy = 57 # mapping speed in uJy/beam root hour
mapping_speed_uK = 74 # mapping speed in uK root hour
K_to_J = (mapping_speed_uJy/mapping_speed_uK) # conversion of Kelvin to Janskys

## Function for calculating the observing time for a single cluster

We calculate the required observing time using the radiometer equation:

$t \propto 1/\sigma^2$ 

where $t$ is observing time in hours and $\sigma$ is the sensitivity/RMS/noise of the observation.

We can set up a proportional relationship with the radiometer equation such that 

$t_2/t_1 \propto (\sigma_1/\sigma_2)^2$ 

where $t_2$ is the required integration time that you are solving for, $\sigma_1$ is the RMS corresponding to your desired map size (see https://gbtdocs.readthedocs.io/en/latest/references/receivers/mustang2/mustang2_mapping.html#mustang-2-mapping-information), $t_1=1$ as the mapping speed are in within 1 hour, and $\sigma_2$ is your desired sensitivity. 

Note that using the fact that mapping speed = $ms = \sigma_1 * (t_1)^{1/2}$ and the radiometer equation in a proportion like the above you can derive that $t_2 = (ms / \sigma_2)^2$.

In [4]:
def calculate_observing_time(M500,z,SNR_required):
    print("-"*5,"Cluster Properties","-"*5)
    print("M500 (1e14):",(M500/1e14).value)
    print("Redshift:",z)

    # Create expected signal map
    ymap = WH.make_A10Map(M500,z,pixsize=pixsize,Dist=True) # create compton-y map
    mymap = WH.smooth_by_M2_beam(ymap,pixsize=pixsize) # convolve signal map (y) with MUSTANG-2 beam

    # Filter signal map
    tab = WH.get_xfertab(scansize) # 2D array of values used to create filtered map
    filtered_model_ymap = FI.apply_xfer(mymap,tab,pixsize) # apply transfer function to compton-y map and create a filtered map
    snr_smoothing_kernel = pix_sigma*0.9 # when an SNR map is created there is smoothing applied - it is recommended to keep this smoothing at 90% of M2 beam (so keep 0.9)
    yxfer = scipy.ndimage.gaussian_filter(filtered_model_ymap, snr_smoothing_kernel)
    
    # Convert model map to observable values (uJy/beam or uK)
    uKmap      = yxfer*y2k*1e6 # convert compton-y map to uK (1e6 is to convert from K to uK)
    uJymap     = uKmap * K_to_J
    
    # Calculate peak value wrt the scale of interest (some values are negative)
    print("-"*5,"Signal Peak Stats","-"*5)
    SZpeak_yraw  = np.max(ymap) # peak of y-map
    print("Unfiltered y peak:", SZpeak_yraw)
    SZpeak_yconvolved  = np.max(mymap)
    print("Beam-convolved y peak:", SZpeak_yconvolved)
    SZpeak_yxfer  = np.max(yxfer)
    print("Filtered y peak:", SZpeak_yxfer)
    SZpeak_uK  = np.min(uKmap) # this is negative so calculate minimum
    print("Filtered peak, uK:", SZpeak_uK)
    SZpeak_uJy = np.min(uJymap) # this is negative so calculate minimum
    print("Filtered peak, uJy/beam:", SZpeak_uJy)
    
    # Calculate the observing time based on the Compton-y peak in uJy/beam
    print("-"*5,"Telescope Time Stats","-"*5)
    target_sensitivity = SZpeak_uJy/SNR_required # calculate the detection threshold based on the required SNR
    print("Your target sensitivity or 1-sigma level in uJy/beam is:",np.abs(target_sensitivity))
    # from the radiometer and a proportional relationship (see math in comments above) we can calculate the observing time required
    ObsTime = np.round((mapping_speed_uJy/target_sensitivity)**2,5) # from unit analysis this variable is in units of hours
    print("On-source time to",str(SNR_required),"sigma peak detection (hrs):",ObsTime)
    TotalTimeRequest = ObsTime * 2 # include the required GBT high frequency factor of 2 to account for overheads
    # GBT time gets scheduled in units of 0.25 hr so convert and 
    TelTime = np.ceil(TotalTimeRequest*4)/4.0

    print("Total telescope time for this cluster (hrs):",TelTime)
    
    return TelTime

## Calculate the telescope time for one cluster

In [5]:
# Physical parameters of your cluster
M500 = 3*1e14*u.M_sun # Mass of cluster
z = 1.0 # The redshift of the cluster
obstime_single_cluster = calculate_observing_time(M500=M500,z=z,SNR_required=7)

----- Cluster Properties -----
M500 (1e14): 3.0
Redshift: 1.0
----- Signal Peak Stats -----
Unfiltered y peak: 9.619897496889924e-05
Beam-convolved y peak: 8.860120011199452e-05
Filtered y peak: 4.829389221185476e-05
Filtered peak, uK: -159.3698442991207
Filtered peak, uJy/beam: -122.7578530412146
----- Telescope Time Stats -----
Your target sensitivity or 1-sigma level in uJy/beam is: 17.536836148744943
On-source time to 7 sigma peak detection (hrs): 10.56446
Total telescope time for this cluster (hrs): 21.25


Why do we require the SNR=7? Based on experience the MUSTANG-2 instrument team highly recommends that you adopt a $7\sigma$ threshold, but depending on your exact science objective, a different threshold may be adopted. Feel free to discuss this with the instrument team.  

## Calculate the telescope time for multiple clusters

In [6]:
# Physical parameters of your cluster
M500s = np.array([1,2,3,4,5,6,7,8])*1e14*u.M_sun # Mass of clusters
zs = np.array([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1]) # Matching redshifts for the M500s list

In [7]:
TotalTime = 0 # Initiate total time counter
for M500,z in zip(M500s,zs):
    print("="*50)
    obstime_cluster = calculate_observing_time(M500=M500,z=z,SNR_required=7)
    TotalTime += obstime_cluster    
print("="*50)
print("Total Telescope Time (hrs): ", TotalTime)

----- Cluster Properties -----
M500 (1e14): 1.0
Redshift: 1.0
----- Signal Peak Stats -----
Unfiltered y peak: 2.7103546504401935e-05
Beam-convolved y peak: 2.4135944165655142e-05
Filtered y peak: 1.4965757817167562e-05
Filtered peak, uK: -49.387000796652956
Filtered peak, uJy/beam: -38.04133845147592
----- Telescope Time Stats -----
Your target sensitivity or 1-sigma level in uJy/beam is: 5.434476921639417
On-source time to 7 sigma peak detection (hrs): 110.01052
Total telescope time for this cluster (hrs): 220.25
----- Cluster Properties -----
M500 (1e14): 2.0
Redshift: 1.0
----- Signal Peak Stats -----
Unfiltered y peak: 6.0365334122178264e-05
Beam-convolved y peak: 5.530305199658348e-05
Filtered y peak: 3.1817993020714536e-05
Filtered peak, uK: -104.99937696835796
Filtered peak, uJy/beam: -80.87789847562708
----- Telescope Time Stats -----
Your target sensitivity or 1-sigma level in uJy/beam is: 11.553985496518154
On-source time to 7 sigma peak detection (hrs): 24.33807
Total teles